# Lab 6: CNN
Mario Dorado Martínez

## Introducción

Des del zoo de Luthadel l'heroi de les Eras ens ha contactat al CITM per demanar-nos que dissenyem una app que identifiqui els animals i generi preguntes perquè els nens puguin aprendre i conèixer més de l’animal que estan veient en temps real. 

Per fer aquesta aplicació s’ha planificat la següent arquitectura:

![image.png](diagram.png)

1. L'usuari fará una fotografia amb el seu dispositiu mòbil i la pujarà a la nostra aplicació
2. El model de CNN detectarà quin animal es tracta.
3. L'animal detectat se li enviarà al model de xat Llama 3.2 perquè ens generi diverses
preguntes amb 4 opcions cada una i només una certa.
4. Un cop generades aquestes preguntes seran mostrades a l’app per poder jugar.

Per aquesta pràctica es demana que es realitzi els punts 1 i 2 d’aquesta primera fase del desenvolupament del videojoc. Per poder entrenar el model haureu de fer servir el conjunt de dades que trobareu disponible en el següent link: [Kaggle Animals Dataset](https://www.kaggle.com/datasets/iamsouravbanerjee/animal-image-dataset-90-different-animals/code)

## Set Up the Enviroment
```bash
pip install tensorflow flask numpy pillow
```

## Load and Preprocess Data
[Kaggle Animals Dataset](https://www.kaggle.com/datasets/iamsouravbanerjee/animal-image-dataset-90-different-animals/code)

## Build and Train the CNN Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load and preprocess data
data_dir = 'path_to_your_dataset'
batch_size = 32
img_height = 180
img_width = 180

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Build the model
model = models.Sequential([
    layers.InputLayer(input_shape=(img_height, img_width, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Save the model
model.save('animal_classifier_model.h5')

## Create an Image Upload Interface

We'll use Flask to create a simple web interface for uploading images.

In [ ]:
from flask import Flask, request, render_template
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

app = Flask(__name__)
model = load_model('animal_classifier_model.h5')
class_names = list(train_generator.class_indices.keys())

def predict_animal(img_path):
    img = image.load_img(img_path, target_size=(img_height, img_width))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    predictions = model.predict(img_array)
    predicted_class = class_names[np.argmax(predictions)]
    return predicted_class

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        file = request.files['file']
        if file:
            file_path = os.path.join('uploads', file.filename)
            file.save(file_path)
            predicted_class = predict_animal(file_path)
            return render_template('result.html', animal=predicted_class)
    return render_template('upload.html')

if __name__ == '__main__':
    app.run(debug=True)

## Create HTML Templates

Create two HTML files: `upload.html` and `result.html`

**upload.html**
```html
<!DOCTYPE html>
<html>
<head>
    <title>Upload Image</title>
</head>
<body>
    <h1>Upload an Image of an Animal</h1>
    <form action="/" method="post" enctype="multipart/form-data">
        <input type="file" name="file">
        <input type="submit" value="Upload">
    </form>
</body>
</html>
```

**result.html**
```html
<!DOCTYPE html>
<html>
<head>
    <title>Result</title>
</head>
<body>
    <h1>Predicted Animal: {{ animal }}</h1>
    <a href="/">Upload another image</a>
</body>
</html>
```

## Running the Application

1. Save the Python script as app.py

2. Create a directory named `uploads` in the same directory as `app.py`.

3. Run the Flask application:

```bash
python app.py
```

4. Open your web browser and go to `http://127.0.0.1:5000/`.